In [30]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, Flatten, Dense, Softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [31]:
df = open("C:/Users/owen/Desktop/Training Dataset/dan_train.csv").read().splitlines()
games = [i.split(',',2)[-1] for i in df]

In [32]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}

In [33]:
def prepare_input(moves):
    x = np.zeros((19,19,4))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,3] = 1
    x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    return x

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

In [34]:
n_games = 0
n_moves = 0
for game in games:
    n_games += 1
    moves_list = game.split(',')
    for move in moves_list:
        n_moves += 1

In [38]:
class MyDataGenerator(Sequence):
    def __init__(self, games, n_classes, batch_size=32, shuffle=True):
        self.games = games
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.games))
        self.n_classes = n_classes
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.games) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_games = [self.games[i] for i in indexes]
        x, y = self.__data_generation(batch_games)
        return x, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, batch_games):
        x = []
        y = []
        for game in batch_games:
            moves_list = game.split(',')
            for count, move in enumerate(moves_list):
                x.append(prepare_input(moves_list[:count]))
                y.append(prepare_label(moves_list[count]))
        x = np.array(x)
        y = np.array(y)
        return x, keras.utils.to_categorical(y, num_classes=self.n_classes)


In [41]:
n_classes = 10  # 替换为实际的类别数
data_generator = MyDataGenerator(games, n_classes)